In [1]:
%load_ext autoreload
%autoreload 2

import os
import time
import sys

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from metadata import *
from data_manager import *
from distributed_utilities import *
from preprocess_utilities import *

Setting environment for AWS compute node


No vtk
File does not exist: /shared/CSHL_data_processed/MD635/MD635_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_sorted_filenames.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD635/MD635_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD657/MD657_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD657/MD657_sorted_filenames.txt
File does not exist: /shared/CSHL_data_processed/MD657/MD657_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD657/MD657_cropbox.txt


In [2]:
tb_fmt = 'png'

In [5]:
for stack in ['MD652', 'MD653']:
    transfer_data_synced(os.path.join('CSHL_data', stack), 
                     from_hostname='s3raw', to_hostname='ec2', is_dir=True, include_only='*.'+tb_fmt)

    transfer_data_synced(os.path.join('CSHL_data_processed', stack, stack + '_sorted_filenames.txt'), 
                     from_hostname='s3', to_hostname='ec2', is_dir=False)
    
    ########### Align ###########
    
    _, sections_to_filenames = DataManager.load_sorted_filenames(stack=stack)
    valid_filenames = [fn for fn in sections_to_filenames.values() if not is_invalid(fn=fn)]
    
    script = os.path.join(REPO_DIR, 'preprocess', 'align_consecutive_v2.py')
    input_dir = os.path.join(RAW_DATA_DIR, stack)
    output_dir = os.path.join(DATA_DIR, stack, stack + '_elastix_output')
    
    ! rm -r $output_dir
    
    t = time.time()
    print 'Align...'

    run_distributed("%(script)s %(stack)s %(input_dir)s %(output_dir)s \'%%(kwargs_str)s\' %(fmt)s" % \
                    {'script': script,
                    'stack': stack,
                    'input_dir': input_dir,
                    'output_dir': output_dir,
                    'fmt': tb_fmt},
                    kwargs_list=[{'prev_fn': valid_filenames[i-1], 'curr_fn': valid_filenames[i]} for i in range(1, len(valid_filenames))],
                    argument_type='list',
                   cluster_size=16,
                   jobs_per_node=16) # 

    wait_qsub_complete()

    print 'done in', time.time() - t, 'seconds' # 252 seconds
    
    transfer_data_synced(os.path.join('CSHL_data_processed', stack, stack + '_elastix_output'), 
                     from_hostname='ec2', to_hostname='s3', is_dir=True)
    
    ! tar -C /shared/CSHL_data_processed/{stack}/ -cf /shared/CSHL_data_processed/{stack}/{stack}_elastix_output.tar {stack}_elastix_output/
    
    transfer_data_synced(os.path.join('CSHL_data_processed', stack, stack + '_elastix_output.tar'), 
                     from_hostname='ec2', to_hostname='s3', is_dir=False)

aws s3 cp --recursive s3://mousebrainatlas-rawdata/CSHL_data/MD652 /shared/CSHL_data/MD652 --exclude "*" --include "*.png"


Child returned 0
4.55 seconds.
Child returned 0


rm -rf /shared/CSHL_data_processed/MD652/MD652_sorted_filenames.txt && mkdir -p /shared/CSHL_data_processed/MD652
aws s3 cp s3://mousebrainatlas-data/CSHL_data_processed/MD652/MD652_sorted_filenames.txt /shared/CSHL_data_processed/MD652/MD652_sorted_filenames.txt
rm: cannot remove '/shared/CSHL_data_processed/MD652/MD652_elastix_output': No such file or directory


Child returned 0
0.51 seconds.
16 nodes requested, 16 nodes available...Continuing


Align...


...

Child returned 0
0.45 seconds.
16 nodes requested, 16 nodes available...Continuing


Align...


Jobs submitted. Use wait_qsub_complete() to check if they finish.
qsub returned.


done in 397.660309076 seconds
aws s3 cp --recursive /shared/CSHL_data_processed/MD653/MD653_elastix_output s3://mousebrainatlas-data/CSHL_data_processed/MD653/MD653_elastix_output


Child returned 0
21.23 seconds.


- download `elastix_output/` to local machine, edit consecutive transforms in local GUI, generate `custom_transforms/` to S3, upload to S3.
- determine anchor image, upload `anchor.txt` to S3.

In [ ]:
for stack in ['MD652', 'MD653']:
# for stack in ['MD642']:
    
    _, sections_to_filenames = DataManager.load_sorted_filenames(stack=stack)
    valid_filenames = [fn for fn in sections_to_filenames.values() if not is_invalid(fn=fn)]
    
    ########### Compose #############
    
    transfer_data_synced(os.path.join('CSHL_data_processed', stack, stack + '_custom_transforms'), 
                     from_hostname='s3', to_hostname='ec2', is_dir=True)
    
    transfer_data_synced(os.path.join('CSHL_data_processed', stack, stack + '_anchor.txt'), 
                     from_hostname='s3', to_hostname='ec2', is_dir=False)
    
    anchor_fn = DataManager.load_anchor_filename(stack=stack)
    
    script = os.path.join(REPO_DIR, 'preprocess', 'compose_transform_thumbnail_v2.py')
    input_dir = os.path.join(DATA_DIR, stack, stack + '_elastix_output')
    output_fn = os.path.join(DATA_DIR, stack, '%(stack)s_transformsTo_%(anchor_fn)s.pkl' % \
                                                    dict(stack=stack, anchor_fn=anchor_fn))

    ! rm -f *{transformTo}*.pkl
    
    t = time.time()
    print 'Composing transform...'

    run_distributed("%(script)s %(stack)s %(input_dir)s \'%%(kwargs_str)s\' %(anchor_idx)d %(output_fn)s" % \
                {'stack': stack,
                'script': script,
                'input_dir': input_dir,
                'anchor_idx': valid_filenames.index(anchor_fn),
                'output_fn': output_fn},
                kwargs_list=[{'filenames': valid_filenames}],
                argument_type='list',
                   cluster_size=1,
                   jobs_per_node=1)

    linked_name = os.path.join(DATA_DIR, stack, '%(stack)s_transformsTo_anchor.pkl' % dict(stack=stack))
    execute_command('rm -f ' + linked_name)
    execute_command('ln -s ' + output_fn + ' ' + linked_name)

    print 'done in', time.time() - t, 'seconds'
    
    transfer_data_synced(os.path.join('CSHL_data_processed', stack), 
                     from_hostname='ec2', to_hostname='s3', is_dir=True, include_only='*.pkl')
    
#     ########## Warp #############
    
    if stack in all_nissl_stacks:
        pad_bg_color = 'white'
    else:
        pad_bg_color = 'auto'
        
    script = os.path.join(REPO_DIR, 'preprocess', 'warp_crop_IM_v2.py')
    input_dir = os.path.join(RAW_DATA_DIR, stack)
    out_dir = os.path.join(DATA_DIR, stack, stack + '_thumbnail_alignedTo_' + anchor_fn)
    
    ! rm -rf $out_dir
    
    t = time.time()
    print 'Warping...'

    transforms_filename = os.path.join(DATA_DIR, stack, '%(stack)s_transformsTo_%(anchor_fn)s.pkl' % \
                                       dict(stack=stack, anchor_fn=anchor_fn))
    transforms_to_anchor = pickle.load(open(transforms_filename, 'r'))

    if pad_bg_color == 'auto':
        run_distributed('%(script)s %(stack)s %(input_dir)s %(out_dir)s %%(transform)s %%(filename)s %%(output_fn)s thumbnail 0 0 2000 1500 %%(pad_bg_color)s' % \
                        {'script': script,
                        'stack': stack,
                        'input_dir': input_dir,
                        'out_dir': out_dir
                        },
                        kwargs_list=[{'transform': ','.join(map(str, transforms_to_anchor[fn].flatten())),
                                    'filename': fn + '.' + tb_fmt,
                                    'output_fn': fn + '_thumbnail_alignedTo_' + anchor_fn + '.tif',
                                    'pad_bg_color': 'black' if fn.split('-')[1][0] == 'F' else 'white'}
                                    for fn in valid_filenames],
                        exclude_nodes=[33],
                        argument_type='single',
                       cluster_size=16,
                       jobs_per_node=8)
    else:
        run_distributed('%(script)s %(stack)s %(input_dir)s %(out_dir)s %%(transform)s %%(filename)s %%(output_fn)s thumbnail 0 0 2000 1500 %(pad_bg_color)s' % \
                        {'script': script,
                        'stack': stack,
                        'input_dir': input_dir,
                        'out_dir': out_dir,
                        'pad_bg_color': pad_bg_color},
                        kwargs_list=[{'transform': ','.join(map(str, transforms_to_anchor[fn].flatten())),
                                    'filename': fn + '.' + tb_fmt,
                                    'output_fn': fn + '_thumbnail_alignedTo_' + anchor_fn + '.tif'}
                                    for fn in valid_filenames],
                        exclude_nodes=[33],
                        argument_type='single',
                       cluster_size=16,
                       jobs_per_node=8)
        
    wait_qsub_complete()

    print 'done in', time.time() - t, 'seconds' # 266 seconds (aws)
    
    transfer_data_synced(os.path.join('CSHL_data_processed', stack, stack + '_thumbnail_alignedTo_' + anchor_fn),
                    from_hostname='ec2',
                    to_hostname='s3',
                    is_dir=True)

- Go to local GUI, download aligned images to check correctness
- Place cropbox.

In [3]:
for stack in ['MD642', 'MD652', 'MD653']:

    _, sections_to_filenames = DataManager.load_sorted_filenames(stack=stack)
    valid_filenames = [fn for fn in sections_to_filenames.values() if not is_invalid(fn=fn)]
    
    anchor_fn = DataManager.load_anchor_filename(stack=stack)

    ############ Crop ###############
    
    transfer_data_synced(os.path.join('CSHL_data_processed', stack, stack + '_cropbox.txt'), 
                     from_hostname='s3', to_hostname='ec2', is_dir=False)
    
    xmin, xmax, ymin, ymax, first_sec, last_sec = DataManager.load_cropbox(stack=stack)
    w = xmax + 1 - xmin
    h = ymax + 1 - ymin
    x = xmin
    y = ymin
    
    first_fn = sections_to_filenames[first_sec]
    last_fn = sections_to_filenames[last_sec]

    first_idx_among_valid = valid_filenames.index(first_fn)
    last_idx_among_valid = valid_filenames.index(last_fn)
    
    input_dir = os.path.join(DATA_DIR, stack, "%(stack)s_thumbnail_alignedTo_%(anchor_fn)s" % \
                           {'stack': stack, 'anchor_fn': anchor_fn})
    output_dir = os.path.join(DATA_DIR, stack, "%(stack)s_thumbnail_alignedTo_%(anchor_fn)s_cropped" % \
                               {'stack': stack, 'anchor_fn': anchor_fn})
    execute_command('mkdir -p ' + output_dir)
    
    t = time.time()
    sys.stderr.write('cropping thumbnail...')

    execute_command('mogrify -set filename:name %%t -crop %(w)dx%(h)d+%(x)d+%(y)d -write "%(output_dir)s/%%[filename:name]_cropped.tif" %(input_dir)s/*.tif' % \
        {'input_dir': input_dir,
         'output_dir': output_dir,
        'w':w, 'h':h, 'x':x, 'y':y})

    sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 100 seconds
    
    transfer_data_synced(os.path.join('CSHL_data_processed', stack, stack + '_thumbnail_alignedTo_' + anchor_fn + '_cropped'),
                    from_hostname='ec2',
                    to_hostname='s3',
                    is_dir=True)

rm -rf /shared/CSHL_data_processed/MD642/MD642_cropbox.txt && mkdir -p /shared/CSHL_data_processed/MD642
aws s3 cp s3://mousebrainatlas-data/CSHL_data_processed/MD642/MD642_cropbox.txt /shared/CSHL_data_processed/MD642/MD642_cropbox.txt


Child returned 0
Child returned 0
0.45 seconds.
Child returned 0
cropping thumbnail...

mkdir -p /shared/CSHL_data_processed/MD642/MD642_thumbnail_alignedTo_MD642-F53-2017.01.14-12.23.43_MD642_1_0157_cropped
mogrify -set filename:name %t -crop 897x487+576+313 -write "/shared/CSHL_data_processed/MD642/MD642_thumbnail_alignedTo_MD642-F53-2017.01.14-12.23.43_MD642_1_0157_cropped/%[filename:name]_cropped.tif" /shared/CSHL_data_processed/MD642/MD642_thumbnail_alignedTo_MD642-F53-2017.01.14-12.23.43_MD642_1_0157/*.tif


Child returned 0
done in 40.003121 seconds


aws s3 cp --recursive /shared/CSHL_data_processed/MD642/MD642_thumbnail_alignedTo_MD642-F53-2017.01.14-12.23.43_MD642_1_0157_cropped s3://mousebrainatlas-data/CSHL_data_processed/MD642/MD642_thumbnail_alignedTo_MD642-F53-2017.01.14-12.23.43_MD642_1_0157_cropped


...

Child returned 0
0.42 seconds.
Child returned 0
cropping thumbnail...

mkdir -p /shared/CSHL_data_processed/MD653/MD653_thumbnail_alignedTo_MD653-F64-2016.12.21-12.03.55_MD653_2_0191_cropped
mogrify -set filename:name %t -crop 802x525+633+187 -write "/shared/CSHL_data_processed/MD653/MD653_thumbnail_alignedTo_MD653-F64-2016.12.21-12.03.55_MD653_2_0191_cropped/%[filename:name]_cropped.tif" /shared/CSHL_data_processed/MD653/MD653_thumbnail_alignedTo_MD653-F64-2016.12.21-12.03.55_MD653_2_0191/*.tif


Child returned 0
done in 41.535591 seconds


aws s3 cp --recursive /shared/CSHL_data_processed/MD653/MD653_thumbnail_alignedTo_MD653-F64-2016.12.21-12.03.55_MD653_2_0191_cropped s3://mousebrainatlas-data/CSHL_data_processed/MD653/MD653_thumbnail_alignedTo_MD653-F64-2016.12.21-12.03.55_MD653_2_0191_cropped


Child returned 0
3.81 seconds.


In [ ]:
for stack in ['MD652', 'MD653']:
# for stack in ['MD642']:

    _, sections_to_filenames = DataManager.load_sorted_filenames(stack=stack)
    valid_filenames = [fn for fn in sections_to_filenames.values() if not is_invalid(fn=fn)]
    
    xmin, xmax, ymin, ymax, first_sec, last_sec = DataManager.load_cropbox(stack=stack)
    w = xmax + 1 - xmin
    h = ymax + 1 - ymin
    x = xmin
    y = ymin
    
    first_fn = sections_to_filenames[first_sec]
    last_fn = sections_to_filenames[last_sec]

    first_idx_among_valid = valid_filenames.index(first_fn)
    last_idx_among_valid = valid_filenames.index(last_fn)
    
    ########## Expand lossless JPEG2000 ##########

    t = time.time()
    sys.stderr.write('expanding...')

    output_dir = os.path.join('/scratch', 'CSHL_data_processed', stack, stack + '_lossless_tif')
    input_dir = os.path.join('/scratch', 'CSHL_data', stack)
    
    
    run_distributed('rm -r %(input_dir)s' % {'input_dir': input_dir},
                    argument_type='single',
                   cluster_size=16)
    wait_qsub_complete()

    run_distributed('rm -rf %(output_dir)s; mkdir -p %(output_dir)s' % {'output_dir': output_dir},
                        argument_type='single',
                       cluster_size=16)
    wait_qsub_complete()
    

    filenames_to_expand = [fn for fn in valid_filenames[first_idx_among_valid:last_idx_among_valid+1] 
                           if not os.path.exists(os.path.join(input_dir, fn + '_lossless.tif'))]

    download_cmd = 'aws s3 cp s3://mousebrainatlas-rawdata/%(s3_dir)s/%%(fn)s_lossless.jp2 %(input_dir)s/ &&' % \
            {'s3_dir': relative_to_ec2(input_dir, '/scratch'), 'input_dir': input_dir}
        
    expand_cmd = 'export LD_LIBRARY_PATH=/home/ubuntu/KDU79_Demo_Apps_for_Linux-x86-64_170108:$LD_LIBRARY_PATH; %(kdu_bin)s -i %(input_dir)s/%%(fn)s_lossless.jp2 -o %(output_dir)s/%%(fn)s_lossless.tif &&' % \
                    {'kdu_bin': KDU_EXPAND_BIN,
                     'output_dir': output_dir,
                    'input_dir': input_dir}

    upload_cmd = 'aws s3 cp %(output_dir)s/%%(fn)s_lossless.tif s3://mousebrainatlas-data/%(s3_dir)s/ &&' % \
            {'output_dir': output_dir, 's3_dir': relative_to_ec2(output_dir, '/scratch')}

    cleanup_cmd = 'rm -f %(input_dir)s/%%(fn)s_lossless.jp2; rm -f %(output_dir)s/%%(fn)s_lossless.tif' % \
            {'input_dir': input_dir, 'output_dir': output_dir}
        
    run_distributed(download_cmd + expand_cmd + upload_cmd + cleanup_cmd,
                     kwargs_list={'fn': filenames_to_expand},
                    argument_type='single',
                   cluster_size=16)

    wait_qsub_complete()

    sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 1400 seconds.

expanding...16 nodes requested, 16 nodes available...Continuing
Jobs submitted. Use wait_qsub_complete() to check if they finish.
qsub returned.
16 nodes requested, 16 nodes available...Continuing
Jobs submitted. Use wait_qsub_complete() to check if they finish.
qsub returned.
16 nodes requested, 16 nodes available...Continuing
Jobs submitted. Use wait_qsub_complete() to check if they finish.


In [ ]:
for stack in ['MD642', 'MD652', 'MD653']:
# for stack in ['MD642']:
    
    ############# Load prerequisites ###############
    
    t = time.time()
    sys.stderr.write('warp and crop lossless...')
    
    
    transfer_data_synced(os.path.join('CSHL_data_processed', stack, stack + '_sorted_filenames.txt'), 
                     from_hostname='s3', to_hostname='ec2', is_dir=False)
    _, sections_to_filenames = DataManager.load_sorted_filenames(stack=stack)
    valid_filenames = [fn for fn in sections_to_filenames.values() if not is_invalid(fn=fn)]
    
    transfer_data_synced(os.path.join('CSHL_data_processed', stack, stack + '_anchor.txt'), 
                 from_hostname='s3', to_hostname='ec2', is_dir=False)
    anchor_fn = DataManager.load_anchor_filename(stack=stack)

    transfer_data_synced(os.path.join('CSHL_data_processed', stack, stack + '_cropbox.txt'), 
                     from_hostname='s3', to_hostname='ec2', is_dir=False)
    xmin, xmax, ymin, ymax, first_sec, last_sec = DataManager.load_cropbox(stack=stack)
    w = xmax + 1 - xmin
    h = ymax + 1 - ymin
    x = xmin
    y = ymin
    
    first_fn = sections_to_filenames[first_sec]
    last_fn = sections_to_filenames[last_sec]

    first_idx_among_valid = valid_filenames.index(first_fn)
    last_idx_among_valid = valid_filenames.index(last_fn)
    
    transfer_data_synced(os.path.join('CSHL_data_processed', stack), 
                 from_hostname='s3', to_hostname='ec2', is_dir=True, include_only='*.pkl')
    
    tf_filepath = os.path.join(DATA_DIR, stack, '%(stack)s_transformsTo_anchor.pkl' % {'stack':stack})
    tfs = pickle.load(open(tf_filepath, 'r'))
    
    ############# Warp and crop lossless ###############
    
    script_fp = os.path.join(REPO_DIR, 'preprocess', 'warp_crop_IM_v2.py')
    
    input_dir = os.path.join('/scratch', 'CSHL_data_processed', stack, stack + '_lossless_tif')
    output_dir = os.path.join('/scratch', 'CSHL_data_processed', stack, stack + '_lossless_alignedTo_' + anchor_fn + '_cropped')
    
    run_distributed('rm -r %(input_dir)s' % {'input_dir': input_dir},
                    argument_type='single',
                   cluster_size=16)
    wait_qsub_complete()

    run_distributed('rm -rf %(output_dir)s; mkdir -p %(output_dir)s' % {'output_dir': output_dir},
                        argument_type='single',
                       cluster_size=16)
    wait_qsub_complete()
    
    if stack in all_nissl_stacks:
        pad_bg_color = 'white'
    else:
        pad_bg_color = 'auto'
    
    t = time.time()
    sys.stderr.write('warping and cropping lossless...')
    
    download_cmd = 'aws s3 cp s3://mousebrainatlas-data/%(s3_dir)s/%%(filename)s %(input_dir)s/' % \
            {'s3_dir': relative_to_ec2(input_dir, '/scratch'), 'input_dir': input_dir}
    
    if pad_bg_color == 'auto':
        work_cmd = '%(script_path)s %(stack)s %(input_dir)s %(output_dir)s %%(transform)s %%(filename)s %%(output_fn)s lossless %(x)d %(y)d %(w)d %(h)d %%(pad_bg_color)s'%\
                    {'script_path': script_fp,
                    'stack': stack,
                    'input_dir': input_dir,
                    'output_dir': output_dir,
                    'x': x,
                    'y': y,
                    'w': w,
                    'h': h}
    else:
        work_cmd = '%(script_path)s %(stack)s %(input_dir)s %(output_dir)s %%(transform)s %%(filename)s %%(output_fn)s lossless %(x)d %(y)d %(w)d %(h)d %(pad_bg_color)s'%\
                    {'script_path': script_fp,
                    'stack': stack,
                    'input_dir': input_dir,
                    'output_dir': output_dir,
                    'pad_bg_color': pad_bg_color,
                    'x': x,
                    'y': y,
                    'w': w,
                    'h': h}
    
    upload_cmd = 'aws s3 cp %(output_dir)s/%%(output_fn)s s3://mousebrainatlas-data/%(s3_dir)s/' % \
            {'output_dir': output_dir, 's3_dir': relative_to_ec2(output_dir, '/scratch'), 'anchor_fn': anchor_fn}
        
    cleanup_cmd = 'rm -f %(input_dir)s/%%(filename)s; rm -f %(output_dir)s/%%(output_fn)s' % \
            {'input_dir': input_dir, 'output_dir': output_dir}
    
    if pad_bg_color == 'auto':
        run_distributed('&&'.join([download_cmd, work_cmd, upload_cmd, cleanup_cmd]),
                        kwargs_list=[{'transform': ','.join(map(str, tfs[fn].flatten())),
                                    'filename': fn + '_lossless.tif',
                                    'output_fn': fn + '_lossless_alignedTo_' + anchor_fn + '_cropped.tif',
                                    'pad_bg_color': 'black' if fn.split('-')[1][0] == 'F' else 'white'}
                                    for fn in valid_filenames[first_idx_among_valid:last_idx_among_valid+1]],                        
                        argument_type='single',
                       cluster_size=16,
                       jobs_per_node=2)
    else:
        run_distributed('&&'.join([download_cmd, work_cmd, upload_cmd, cleanup_cmd]),
                     kwargs_list=[{'transform': ','.join(map(str, tfs[fn].flatten())),
                                    'filename': fn + '_lossless.tif',
                                    'output_fn': fn + '_lossless_alignedTo_' + anchor_fn + '_cropped.tif'}
                                    for fn in valid_filenames[first_idx_among_valid:last_idx_among_valid+1]],
                    argument_type='single',
                       cluster_size=16,
                       jobs_per_node=2)

    wait_qsub_complete()
    
    sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 1400 seconds.

warp and crop lossless...Child returned 0


rm -rf /shared/CSHL_data_processed/MD642/MD642_sorted_filenames.txt && mkdir -p /shared/CSHL_data_processed/MD642
aws s3 cp s3://mousebrainatlas-data/CSHL_data_processed/MD642/MD642_sorted_filenames.txt /shared/CSHL_data_processed/MD642/MD642_sorted_filenames.txt


Child returned 0
0.43 seconds.
Child returned 0


rm -rf /shared/CSHL_data_processed/MD642/MD642_anchor.txt && mkdir -p /shared/CSHL_data_processed/MD642
aws s3 cp s3://mousebrainatlas-data/CSHL_data_processed/MD642/MD642_anchor.txt /shared/CSHL_data_processed/MD642/MD642_anchor.txt


Child returned 0
0.42 seconds.
Child returned 0


rm -rf /shared/CSHL_data_processed/MD642/MD642_cropbox.txt && mkdir -p /shared/CSHL_data_processed/MD642
aws s3 cp s3://mousebrainatlas-data/CSHL_data_processed/MD642/MD642_cropbox.txt /shared/CSHL_data_processed/MD642/MD642_cropbox.txt


Child returned 0
0.42 seconds.


aws s3 cp --recursive s3://mousebrainatlas-data/CSHL_data_processed/MD642 /shared/CSHL_data_processed/MD642 --exclude "*" --include "*.pkl"


Child returned 0
2.96 seconds.
16 nodes requested, 16 nodes available...Continuing
Jobs submitted. Use wait_qsub_complete() to check if they finish.
qsub returned.
16 nodes requested, 16 nodes available...Continuing
Jobs submitted. Use wait_qsub_complete() to check if they finish.
qsub returned.
warping and cropping lossless...16 nodes requested, 16 nodes available...Continuing
Jobs submitted. Use wait_qsub_complete() to check if they finish.


In [ ]:
for stack in ['MD642', 'MD652', 'MD653']:
    
    ############# Generate mask ###############
    
    t = time.time()
    sys.stderr.write('warp and crop lossless...')
    
    transfer_data_synced(os.path.join('CSHL_data_processed', stack, stack + '_sorted_filenames.txt'), 
                     from_hostname='s3', to_hostname='ec2', is_dir=False)
    _, sections_to_filenames = DataManager.load_sorted_filenames(stack=stack)
    valid_filenames = [fn for fn in sections_to_filenames.values() if not is_invalid(fn=fn)]
    
    transfer_data_synced(os.path.join('CSHL_data_processed', stack, stack + '_anchor.txt'), 
                 from_hostname='s3', to_hostname='ec2', is_dir=False)
    anchor_fn = DataManager.load_anchor_filename(stack=stack)

    transfer_data_synced(os.path.join('CSHL_data_processed', stack, stack + '_cropbox.txt'), 
                     from_hostname='s3', to_hostname='ec2', is_dir=False)
    xmin, xmax, ymin, ymax, first_sec, last_sec = DataManager.load_cropbox(stack=stack)
    w = xmax + 1 - xmin
    h = ymax + 1 - ymin
    x = xmin
    y = ymin
    
    first_fn = sections_to_filenames[first_sec]
    last_fn = sections_to_filenames[last_sec]

    first_idx_among_valid = valid_filenames.index(first_fn)
    last_idx_among_valid = valid_filenames.index(last_fn)
    
    transfer_data_synced(os.path.join('CSHL_data_processed', stack), 
                 from_hostname='s3', to_hostname='ec2', is_dir=True, include_only='*.pkl')
    
    tf_filepath = os.path.join(DATA_DIR, stack, '%(stack)s_transformsTo_anchor.pkl' % {'stack':stack})
    tfs = pickle.load(open(tf_filepath, 'r'))
    
    ############# Warp and crop lossless ###############
    
    script_fp = os.path.join(REPO_DIR, 'preprocess', 'warp_crop_IM_v2.py')
    
    input_dir = os.path.join('/scratch', 'CSHL_data_processed', stack, stack + '_lossless_tif')
    output_dir = os.path.join('/scratch', 'CSHL_data_processed', stack, stack + '_lossless_alignedTo_' + anchor_fn + '_cropped')
    
    run_distributed('rm -r %(input_dir)s' % {'input_dir': input_dir},
                    argument_type='single',
                   cluster_size=16)
    wait_qsub_complete()

    run_distributed('rm -rf %(output_dir)s; mkdir -p %(output_dir)s' % {'output_dir': output_dir},
                        argument_type='single',
                       cluster_size=16)
    wait_qsub_complete()
    
    if stack in all_nissl_stacks:
        pad_bg_color = 'white'
    else:
        pad_bg_color = 'auto'
    
    t = time.time()
    sys.stderr.write('warping and cropping lossless...')
    
    download_cmd = 'aws s3 cp s3://mousebrainatlas-data/%(s3_dir)s/%%(filename)s %(input_dir)s/' % \
            {'s3_dir': relative_to_ec2(input_dir, '/scratch'), 'input_dir': input_dir}
    
    if pad_bg_color == 'auto':
        work_cmd = '%(script_path)s %(stack)s %(input_dir)s %(output_dir)s %%(transform)s %%(filename)s %%(output_fn)s lossless %(x)d %(y)d %(w)d %(h)d %%(pad_bg_color)s'%\
                    {'script_path': script_fp,
                    'stack': stack,
                    'input_dir': input_dir,
                    'output_dir': output_dir,
                    'x': x,
                    'y': y,
                    'w': w,
                    'h': h}
    else:
        work_cmd = '%(script_path)s %(stack)s %(input_dir)s %(output_dir)s %%(transform)s %%(filename)s %%(output_fn)s lossless %(x)d %(y)d %(w)d %(h)d %(pad_bg_color)s'%\
                    {'script_path': script_fp,
                    'stack': stack,
                    'input_dir': input_dir,
                    'output_dir': output_dir,
                    'pad_bg_color': pad_bg_color,
                    'x': x,
                    'y': y,
                    'w': w,
                    'h': h}
    
    upload_cmd = 'aws s3 cp %(output_dir)s/%%(output_fn)s s3://mousebrainatlas-data/%(s3_dir)s/' % \
            {'output_dir': output_dir, 's3_dir': relative_to_ec2(output_dir, '/scratch'), 'anchor_fn': anchor_fn}
        
    cleanup_cmd = 'rm -f %(input_dir)s/%%(filename)s; rm -f %(output_dir)s/%%(output_fn)s' % \
            {'input_dir': input_dir, 'output_dir': output_dir}
    
    if pad_bg_color == 'auto':
        run_distributed('&&'.join([download_cmd, work_cmd, upload_cmd, cleanup_cmd]),
                        kwargs_list=[{'transform': ','.join(map(str, tfs[fn].flatten())),
                                    'filename': fn + '_lossless.tif',
                                    'output_fn': fn + '_lossless_alignedTo_' + anchor_fn + '_cropped.tif',
                                    'pad_bg_color': 'black' if fn.split('-')[1][0] == 'F' else 'white'}
                                    for fn in valid_filenames[first_idx_among_valid:last_idx_among_valid+1]],                        
                        argument_type='single',
                       cluster_size=16,
                       jobs_per_node=2)
    else:
        run_distributed('&&'.join([download_cmd, work_cmd, upload_cmd, cleanup_cmd]),
                     kwargs_list=[{'transform': ','.join(map(str, tfs[fn].flatten())),
                                    'filename': fn + '_lossless.tif',
                                    'output_fn': fn + '_lossless_alignedTo_' + anchor_fn + '_cropped.tif'}
                                    for fn in valid_filenames[first_idx_among_valid:last_idx_among_valid+1]],
                    argument_type='single',
                       cluster_size=16,
                       jobs_per_node=2)

    wait_qsub_complete()
    
    sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 1400 seconds.

In [26]:
# for stack in ['MD642', 'MD652', 'MD653']:
for stack in ['MD652']:
    
    transfer_data_synced(os.path.join('CSHL_data', stack), 
                         from_hostname='s3raw', to_hostname='ec2', is_dir=True, include_only='*.'+tb_fmt)
    
    transfer_data_synced(os.path.join('CSHL_data_processed', stack, stack + '_sorted_filenames.txt'), 
                     from_hostname='s3', to_hostname='ec2', is_dir=False)
    _, sections_to_filenames = DataManager.load_sorted_filenames(stack=stack)
    valid_filenames = [fn for fn in sections_to_filenames.values() if not is_invalid(fn=fn)]
    
    script = os.path.join(REPO_DIR, 'preprocess', 'generate_thumbnail_masks_v4.py')
    input_dir = os.path.join(RAW_DATA_DIR, stack)
    output_dir = create_if_not_exists(os.path.join(DATA_DIR, stack, stack + '_submasks'))
    ! rm -f output_dir/*
    
    t = time.time()
    print 'Generating thumbnail mask...',

    wait_num_nodes(16)

    run_distributed(command='%(script_path)s %(stack)s %(input_dir)s \'%%(filenames)s\' %(output_dir)s' % \
                    {'script_path': script,
                    'stack': stack,
                    'input_dir': input_dir,
                    'output_dir': output_dir},
                    kwargs_list=dict(filenames=valid_filenames),
                    argument_type='list2',
                   cluster_size=16,
                   jobs_per_node=1)

    wait_qsub_complete()

    print 'done in', time.time() - t, 'seconds' # 300s (aws)
    
    transfer_data_synced(relative_to_ec2(output_dir),
                    from_hostname='ec2',
                    to_hostname='s3',
                    is_dir=True)

aws s3 cp --recursive s3://mousebrainatlas-rawdata/CSHL_data/MD652 /shared/CSHL_data/MD652 --exclude "*" --include "*.png"


Child returned 0
4.63 seconds.
Child returned 0


rm -rf /shared/CSHL_data_processed/MD652/MD652_sorted_filenames.txt && mkdir -p /shared/CSHL_data_processed/MD652
aws s3 cp s3://mousebrainatlas-data/CSHL_data_processed/MD652/MD652_sorted_filenames.txt /shared/CSHL_data_processed/MD652/MD652_sorted_filenames.txt


Child returned 0
0.45 seconds.
All nodes are ready.
Wait for SGE to know all nodes (timeout in 300 seconds)...
All nodes are ready.
16 nodes requested, 16 nodes available...Continuing


Generating thumbnail mask...

Jobs submitted. Use wait_qsub_complete() to check if they finish.
qsub returned.


 done in 533.41121006 seconds
aws s3 cp --recursive /shared/CSHL_data_processed/MD652/MD652_submasks s3://mousebrainatlas-data/CSHL_data_processed/MD652/MD652_submasks


Child returned 0
6.57 seconds.


In [8]:
! rm ~/*.log

In [14]:
! tail ~/stdout_2_0.log


download: s3://mousebrainatlas-data/CSHL_data_processed/MD642/MD642_lossless_tif/MD642-N14-2017.01.18-16.25.58_MD642_3_0042_lossless.tif to ../../scratch/CSHL_data_processed/MD642/MD642_lossless_tif/MD642-N14-2017.01.18-16.25.58_MD642_3_0042_lossless.tif


In [15]:
! tail ~/stderr_2_0.log

rm: cannot remove '/scratch/CSHL_data_processed/MD642/MD642_lossless_tif': No such file or directory
An error occurred (404) when calling the HeadObject operation: Key "CSHL_data_processed/MD642/MD642_lossless_tif/MD642-N14-2017.01.18-16.25.58_MD642_3_0042_lossless.tif" does not exist
rm: cannot remove '/scratch/CSHL_data_processed/MD642/MD642_lossless_tif': No such file or directory
usage: warp_crop_IM_v2.py [-h]
                          stack_name input_dir output_dir transform filename
                          output_fn suffix x y w h background_color
warp_crop_IM_v2.py: error: unrecognized arguments: s3 cp /scratch/CSHL_data_processed/MD642/MD642_lossless_alignedTo_MD642-F53-2017.01.14-12.23.43_MD642_1_0157_cropped/MD642-N14-2017.01.18-16.25.58_MD642_3_0042_lossless_alignedTo_MD642-F53-2017.01.14-12.23.43_MD642_1_0157_cropped.tif s3://mousebrainatlas-data/CSHL_data_processed/MD642/MD642_lossless_alignedTo_MD642-F53-2017.01.14-12.23.43_MD642_1_0157_cropped/
